In [ ]:
df_tr0 = pd.rea
df_tr1 = pd.read_hdf('../../search/feats/tr_station_pass1.hdf5','table')
df_tr2 = pd.read_hdf('../../search/feats/tr_line_pass2.hdf5','table')
df_tr3 = pd.read_hdf('../../search/feats/tr_number_stations_passed3.hdf5','table')
df_tr4 = pd.read_hdf('../../search/feats/tr_nonNaN_full4.hdf5','table')
df_tr5 = pd.read_hdf('../../search/feats/tr_sum_full5.hdf5','table')
df_tr6 = pd.read_hdf('../../search/feats/tr_line_station_nonNaN6.hdf5','table')
df_tr7 = pd.read_hdf('../../search/feats/tr_line_station_sum7.hdf5','table')
df_tr8 = pd.read_hdf('../../search/feats/tr_line_station_minmax8.hdf5','table')
df_tr9 = pd.read_hdf('../../search/feats/tr_minmaxdiff_full_date9.hdf5','table')
df_tr10 = pd.read_hdf('../../search/feats/tr_line_station_diff_date10.hdf5','table')
df_tr13 = pd.read_hdf('../../search/feats/tr_hash_full_date_num_cat13.hdf5','table')

df_train = pd.concat([df_tr1,df_tr2,df_tr3,df_tr4,df_tr5,df_tr6,df_tr7,df_tr8,df_tr9,df_tr10,df_tr13],axis=1)

#del df_tr1,df_tr2,df_tr3,df_tr4,df_tr5,df_tr6,df_tr7,df_tr8,df_tr9,df_tr10,df_tr13
import gc
gc.collect()

df_train.fillna(-9999999.0,inplace=True)

#now, let's find features with no variance 
toKeep = []
toDelete = []
## loop through the DataFrame's columns
for col in df_train:
    ## if the value_counts method returns more than one uniqe entity,
    ## append the column name to 'toKeep'
    if len(df_train[col].value_counts()) > 1:
        toKeep.append(col)
    ## if not, append to 'toDelete'.
    else:
        toDelete.append(col)

df_train.drop(toDelete, 1, inplace = True)

df_train.shape

hashed = df_train.apply(lambda x: hash(tuple(x)), axis = 0)

hashed.unique().shape,hashed.shape

hashed_df = pd.DataFrame(hashed)
hashed_df.columns = ['hash123']
hashed_df['column'] = hashed_df.index
hashed_df.reset_index(drop=True,inplace=True)

hashed_unq = hashed_df.drop_duplicates(subset='hash123',keep='first')

hashed_unq.head(2)

df_train = df_train[hashed_unq.column]

df_train.replace(-9999999.0,np.nan,inplace=1)

df_train.to_hdf('train-rest-542-unq.hdf5','table')

In [16]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
from numba import jit

#@jit
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf==0:
        return 0
    else:
        return sup / np.sqrt(inf)

#@jit
def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true) # number of positive
    numn = n - nump # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    prev_proba = -1
    best_proba = -1
    mccs = np.zeros(n)
    for i in range(n):
        # all items with idx < i are predicted negative while others are predicted positive
        # only evaluate mcc when probability changes
        proba = y_prob[idx[i]]
        if proba != prev_proba:
            prev_proba = proba
            new_mcc = mcc(tp, tn, fp, fn)
            if new_mcc >= best_mcc:
                best_mcc = new_mcc
                best_id = i
                best_proba = proba
        mccs[i] = new_mcc
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
    if show:
        y_pred = (y_prob >= best_proba).astype(int)
        score = matthews_corrcoef(y_true, y_pred)
        print(score, best_mcc)
        plt.plot(mccs)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc

def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', best_mcc

In [13]:
def create_feature_map(features):
    outfile = open('xgb_542_feats.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

from operator import itemgetter
def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb_542_feats.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

In [3]:
df_train = pd.read_hdf('train-rest-542-unq.hdf5','table')

y = pd.read_csv('../../input_orig/train_numeric.csv', usecols=['Response'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42, stratify=y.values)

In [11]:
del df_train
import gc
gc.collect()

dtrain = xgb.DMatrix(X_train, label=y_train)

del X_train
del y_train
gc.collect()

dval = xgb.DMatrix(X_test, label=y_test)

del X_test
del y_test
gc.collect()

7

In [19]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst.predict(dval, ntree_limit=bst.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.699066	eval-auc:0.681639
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.712907	eval-auc:0.68714
[2]	train-auc:0.719075	eval-auc:0.689231
[3]	train-auc:0.72521	eval-auc:0.693082
[4]	train-auc:0.731996	eval-auc:0.69683
[5]	train-auc:0.737334	eval-auc:0.699178
[6]	train-auc:0.740471	eval-auc:0.701829
[7]	train-auc:0.74236	eval-auc:0.702282
[8]	train-auc:0.74628	eval-auc:0.702758
[9]	train-auc:0.750573	eval-auc:0.705855
[10]	train-auc:0.753716	eval-auc:0.706776
[11]	train-auc:0.754963	eval-auc:0.707527
[12]	train-auc:0.758201	eval-auc:0.709812
[13]	train-auc:0.75995	eval-auc:0.710308
[14]	train-auc:0.76333	eval-auc:0.711436
[15]	train-auc:0.764134	eval-auc:0.71165
[16]	train-auc:0.767082	eval-auc:0.713163
[17]	train-auc:0.768599	eval-auc:0.712839
[18]	train-auc:0.770427	eval-auc:0.712738
[19]	train-auc:0.773894	eval-auc:0.714038
[20]	train-auc:0.774163	eval-auc:0.71

In [20]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst1 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst1.predict(dval, ntree_limit=bst1.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst1.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.758841	eval-auc:0.687528
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.794585	eval-auc:0.695611
[2]	train-auc:0.810437	eval-auc:0.702159
[3]	train-auc:0.827551	eval-auc:0.709454
[4]	train-auc:0.837315	eval-auc:0.708422
[5]	train-auc:0.850633	eval-auc:0.712681
[6]	train-auc:0.858197	eval-auc:0.713972
[7]	train-auc:0.869674	eval-auc:0.718241
[8]	train-auc:0.874318	eval-auc:0.71934
[9]	train-auc:0.880495	eval-auc:0.719881
[10]	train-auc:0.885915	eval-auc:0.720362
[11]	train-auc:0.890395	eval-auc:0.72189
[12]	train-auc:0.896532	eval-auc:0.723485
[13]	train-auc:0.907529	eval-auc:0.72393
[14]	train-auc:0.910828	eval-auc:0.724873
[15]	train-auc:0.917204	eval-auc:0.72524
[16]	train-auc:0.920328	eval-auc:0.72671
[17]	train-auc:0.926636	eval-auc:0.726907
[18]	train-auc:0.9291	eval-auc:0.726862
[19]	train-auc:0.931419	eval-auc:0.726387
[20]	train-auc:0.934552	eval-auc:0.7

In [21]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 9,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst2 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1, feval=mcc_eval, maximize=True)

val_pred = bst2.predict(dval, ntree_limit=bst2.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst2.get_score(fmap='', importance_type='gain')))

[0]	train-MCC:0.178308	eval-MCC:0.160293
Multiple eval metrics have been passed: 'eval-MCC' will be used for early stopping.

Will train until eval-MCC hasn't improved in 20 rounds.
[1]	train-MCC:0.191179	eval-MCC:0.164462
[2]	train-MCC:0.199441	eval-MCC:0.165994
[3]	train-MCC:0.204631	eval-MCC:0.167502
[4]	train-MCC:0.212142	eval-MCC:0.168161
[5]	train-MCC:0.216545	eval-MCC:0.170799
[6]	train-MCC:0.220323	eval-MCC:0.172941
[7]	train-MCC:0.225237	eval-MCC:0.174645
[8]	train-MCC:0.229763	eval-MCC:0.177294
[9]	train-MCC:0.234608	eval-MCC:0.17691
[10]	train-MCC:0.235405	eval-MCC:0.178281
[11]	train-MCC:0.240247	eval-MCC:0.177141
[12]	train-MCC:0.244404	eval-MCC:0.177886
[13]	train-MCC:0.24804	eval-MCC:0.178756
[14]	train-MCC:0.248737	eval-MCC:0.178987
[15]	train-MCC:0.250791	eval-MCC:0.177837
[16]	train-MCC:0.254399	eval-MCC:0.180088
[17]	train-MCC:0.256431	eval-MCC:0.181246
[18]	train-MCC:0.257809	eval-MCC:0.182305
[19]	train-MCC:0.259623	eval-MCC:0.183364
[20]	train-MCC:0.262662	eval-MC

In [22]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst3 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1, feval=mcc_eval, maximize=True)

val_pred = bst3.predict(dval, ntree_limit=bst3.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst3.get_score(fmap='', importance_type='gain')))

[0]	train-MCC:0.178308	eval-MCC:0.160293
Multiple eval metrics have been passed: 'eval-MCC' will be used for early stopping.

Will train until eval-MCC hasn't improved in 20 rounds.
[1]	train-MCC:0.195117	eval-MCC:0.16395
[2]	train-MCC:0.209947	eval-MCC:0.162653
[3]	train-MCC:0.231044	eval-MCC:0.164874
[4]	train-MCC:0.241275	eval-MCC:0.167729
[5]	train-MCC:0.248709	eval-MCC:0.176403
[6]	train-MCC:0.25742	eval-MCC:0.17912
[7]	train-MCC:0.26595	eval-MCC:0.181083
[8]	train-MCC:0.274857	eval-MCC:0.181782
[9]	train-MCC:0.282419	eval-MCC:0.186686
[10]	train-MCC:0.285879	eval-MCC:0.18527
[11]	train-MCC:0.291838	eval-MCC:0.184551
[12]	train-MCC:0.301254	eval-MCC:0.184895
[13]	train-MCC:0.307163	eval-MCC:0.188009
[14]	train-MCC:0.314133	eval-MCC:0.187469
[15]	train-MCC:0.318021	eval-MCC:0.187469
[16]	train-MCC:0.324782	eval-MCC:0.187942
[17]	train-MCC:0.332063	eval-MCC:0.187727
[18]	train-MCC:0.334346	eval-MCC:0.187469
[19]	train-MCC:0.336656	eval-MCC:0.187224
[20]	train-MCC:0.343769	eval-MCC:0

In [24]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 18,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst4 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1, feval=mcc_eval, maximize=True)

val_pred = bst4.predict(dval, ntree_limit=bst4.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst4.get_score(fmap='', importance_type='gain')))

[0]	train-MCC:0.179757	eval-MCC:0.158451
Multiple eval metrics have been passed: 'eval-MCC' will be used for early stopping.

Will train until eval-MCC hasn't improved in 20 rounds.
[1]	train-MCC:0.210325	eval-MCC:0.163676
[2]	train-MCC:0.234816	eval-MCC:0.165723
[3]	train-MCC:0.256643	eval-MCC:0.170747
[4]	train-MCC:0.275259	eval-MCC:0.175532
[5]	train-MCC:0.29862	eval-MCC:0.182663
[6]	train-MCC:0.328596	eval-MCC:0.185168
[7]	train-MCC:0.352017	eval-MCC:0.185542
[8]	train-MCC:0.377792	eval-MCC:0.186104
[9]	train-MCC:0.399881	eval-MCC:0.189258
[10]	train-MCC:0.411785	eval-MCC:0.190846
[11]	train-MCC:0.430093	eval-MCC:0.190068
[12]	train-MCC:0.44131	eval-MCC:0.191249
[13]	train-MCC:0.462552	eval-MCC:0.192681
[14]	train-MCC:0.474574	eval-MCC:0.195167
[15]	train-MCC:0.491623	eval-MCC:0.19552
[16]	train-MCC:0.500031	eval-MCC:0.198767
[17]	train-MCC:0.509511	eval-MCC:0.198212
[18]	train-MCC:0.52215	eval-MCC:0.200186
[19]	train-MCC:0.539168	eval-MCC:0.200225
[20]	train-MCC:0.54598	eval-MCC:0

In [25]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 24,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 4
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst5 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1, feval=mcc_eval, maximize=True)

val_pred = bst5.predict(dval, ntree_limit=bst5.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst5.get_score(fmap='', importance_type='gain')))

[0]	train-MCC:0.180182	eval-MCC:0.158451
Multiple eval metrics have been passed: 'eval-MCC' will be used for early stopping.

Will train until eval-MCC hasn't improved in 20 rounds.
[1]	train-MCC:0.221654	eval-MCC:0.162774
[2]	train-MCC:0.263315	eval-MCC:0.166496
[3]	train-MCC:0.29726	eval-MCC:0.169594
[4]	train-MCC:0.330524	eval-MCC:0.176517
[5]	train-MCC:0.365742	eval-MCC:0.183475
[6]	train-MCC:0.399565	eval-MCC:0.187109
[7]	train-MCC:0.437661	eval-MCC:0.187001
[8]	train-MCC:0.47248	eval-MCC:0.189172
[9]	train-MCC:0.495746	eval-MCC:0.191577
[10]	train-MCC:0.514473	eval-MCC:0.194005
[11]	train-MCC:0.536855	eval-MCC:0.194151
[12]	train-MCC:0.5575	eval-MCC:0.194038
[13]	train-MCC:0.58087	eval-MCC:0.195859
[14]	train-MCC:0.60122	eval-MCC:0.198823
[15]	train-MCC:0.61789	eval-MCC:0.200054
[16]	train-MCC:0.629369	eval-MCC:0.202687
[17]	train-MCC:0.644104	eval-MCC:0.203004
[18]	train-MCC:0.657898	eval-MCC:0.202687
[19]	train-MCC:0.668525	eval-MCC:0.20237
[20]	train-MCC:0.675387	eval-MCC:0.20

In [27]:
#get_importance(bst5, bst5.feature_names)

In [28]:
bst5.save_model('xgb_542_0217mcc_0707auc.model')